#Install Library

[RDKit ](https://github.com/rdkit/rdkit)

[DGL](https://github.com/dmlc/dgl/)

[DGL-LifeSci](https://github.com/awslabs/dgl-lifesci)
[link text](https://)

In [3]:
!pip install rdkit-pypi
!pip install dgllife
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 48.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinu

In [4]:
import os
# os.environ['PYTHONHASHSEED'] = '0'
# os.environ['CUDA_VISIBLE_DEVICES']='-1'
# os.environ['TF_CUDNN_USE_AUTOTUNE'] ='0'

import dgl
import sys
import torch
import random
import cv2
import torchvision
import pandas as pd
import numpy as np
import tensorflow as tf
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import torchvision.transforms as transforms

from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit import DataStructs

from torch.utils.data import DataLoader, Dataset
from dgllife.data import SIDER
from numpy import array
from numpy import argmax
from tensorflow.keras.utils import to_categorical
from dgllife.model import load_pretrained
from tensorflow.keras.callbacks import  History
from dgllife.utils import smiles_to_bigraph, CanonicalAtomFeaturizer, AttentiveFPAtomFeaturizer
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm, trange
from sklearn.model_selection import train_test_split
from dgl.data.utils import get_download_dir, download, _get_dgl_url, extract_archive
from dgllife.data.csv_dataset import MoleculeCSVDataset
from dgllife.utils.mol_to_graph import smiles_to_bigraph

# random.seed(46)
# np.random.seed(46)
# tf.random.set_seed(46)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
train_file_path = '/content/drive/My Drive/IND_ability_train.csv'
test_file_path = '/content/drive/MyDrive/IND_ability_test.csv'

In [7]:
df_train =  pd.read_csv(train_file_path, usecols=['smiles', 'label'])
df_test = pd.read_csv(test_file_path , usecols=['smiles', 'label'])

In [8]:
df_train

,smiles,label
0,CCc1ccc(cc1)Nc1[n]c2ccccc2o1,0
1,C[C@H]1NCCO[C@@H]1c1ccccc1,1
2,O=C1NC(=O)c2c1c1c[n]c[n]1c1[nH]c3ccccc3c12,0
3,Cc1[n]c(CCCC)c(Cc2ccc(cc2)-c2ccccc2-c2[nH][n][...,1
4,COc1c(c(F)cc2c1[n](cc(C(O)=O)c2=O)[C@@H]1C[C@@...,1
...,...,...
24592,CN1CCN(CC1)C(=O)Oc1cc2c([C@H](CBr)CN2C(=O)c2cc...,1
24593,C[C@@H](CC)NC[C@H](O)c1cc2CCCCc2cc1,1
24594,CC(C)[C@@H](OC(=O)C[C@@H](O)C1C(=O)OC(=O)C=1C)...,0
24595,COc1c(cc(cc1OC)[C@H]1[C@@H]([C@@H](CO)[C@@H](O...,1


In [9]:
class DATASET(MoleculeCSVDataset):

    def __init__(self,dataFram=None,
                 smiles_to_graph=smiles_to_bigraph,
                 node_featurizer=None,
                 edge_featurizer=None,
                 load=False,
                 log_every=1000,
                 cache_file_path = None,
                 n_jobs=1):


        super(DATASET, self).__init__(df=dataFram,
                                    smiles_to_graph=smiles_to_graph,
                                    node_featurizer=node_featurizer,
                                    edge_featurizer=edge_featurizer,
                                    smiles_column='smiles',
                                    cache_file_path = cache_file_path,
                                    load=load,
                                    log_every=log_every,
                                    init_mask=True,
                                    n_jobs=n_jobs)

    def __getitem__(self, item):

        return self.smiles[item], self.graphs[item], self.labels[item], self.mask[item]

In [10]:
from dgllife.model import load_pretrained
from dgllife.model import MLPPredictor

def get_sider_model (model_name = "GCN_attentivefp_SIDER"):
    if model_name ==  "GCN_attentivefp_SIDER":
        predictor_dropout = 0.08333992387843633
        predictor_hidden_feats = 1024
    else:
        predictor_dropout =  0.034959769945995006
        predictor_hidden_feats = 512

    gcn = load_pretrained(model_name)
    gnn_out_feats = gcn.gnn.hidden_feats[-1]
    gcn.predict = MLPPredictor(2 * gnn_out_feats, predictor_hidden_feats, predictor_hidden_feats, predictor_dropout)

    return gcn

In [11]:
def create_dataset_with_gcn(dataset, GCN ):
    created_data = []
    for i, data in enumerate(dataset):
        smiles, g, label, mask = data
        g = g.to(device)
        g = dgl.add_self_loop(g)
        graph_feats = g.ndata.pop('h')
        embbed = GCN(g, graph_feats)
        embbed = embbed.to('cpu')
        embbed = embbed.detach().numpy()
        d = (embbed, label, smiles)
        created_data.append(d)
    print('Data created!!')
    return created_data

In [12]:
def count_lablel(dataset):

    label_pos = 0
    lable_neg = 0

    for i, data in enumerate(dataset):
        embbed, lbl, smiles = data
        if lbl == 1.:
            label_pos += 1
        else:
            lable_neg += 1

    return label_pos, lable_neg

In [13]:
train_dataset = DATASET(df_train, smiles_to_bigraph, AttentiveFPAtomFeaturizer(), cache_file_path="./sider_dglgraph.bin")
test_dataset = DATASET(df_test, smiles_to_bigraph, AttentiveFPAtomFeaturizer(), cache_file_path="./sider_dglgraph.bin")

Processing dgl graphs from scratch...
Processing molecule 1000/24597
Processing molecule 2000/24597
Processing molecule 3000/24597
Processing molecule 4000/24597
Processing molecule 5000/24597
Processing molecule 6000/24597
Processing molecule 7000/24597
Processing molecule 8000/24597
Processing molecule 9000/24597
Processing molecule 10000/24597
Processing molecule 11000/24597
Processing molecule 12000/24597
Processing molecule 13000/24597
Processing molecule 14000/24597
Processing molecule 15000/24597
Processing molecule 16000/24597
Processing molecule 17000/24597
Processing molecule 18000/24597
Processing molecule 19000/24597
Processing molecule 20000/24597
Processing molecule 21000/24597
Processing molecule 22000/24597
Processing molecule 23000/24597
Processing molecule 24000/24597
Processing dgl graphs from scratch...
Processing molecule 1000/2732
Processing molecule 2000/2732


In [14]:
model_name = 'GCN_attentivefp_SIDER'
gcn_model = get_sider_model(model_name)
gcn_model.eval()
gcn_model = gcn_model.to(device)

GCN_attentivefp_SIDER_pre_trained.pth:   0%|          | 0.00/3.09M [00:00<?, ?B/s]

Pretrained model loaded


In [15]:
train_ds = create_dataset_with_gcn(train_dataset, gcn_model)
test_ds = create_dataset_with_gcn(test_dataset, gcn_model)

Data created!!
Data created!!


In [16]:
label_pos , label_neg = count_lablel(train_ds)
print(f'train positive label: {label_pos} - train negative label: {label_neg}')

label_pos , label_neg = count_lablel(test_ds)
print(f'Test positive label: {label_pos} - Test negative label: {label_neg}')

train positive label: 8442 - train negative label: 16155
Test positive label: 944 - Test negative label: 1788


In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def sequential_model():
    inputs = Input(shape=(1024,))  # Define the input shape

    # Define the network
    L1 = Dense(1024, activation='relu')(inputs)
    L1_D = Dropout(0.2)(L1)

    L2 = Dense(512, activation='relu')(L1_D)
    L2_D = Dropout(0.2)(L2)

    L3 = Dense(256, activation='relu')(L2_D)
    L3_D = Dropout(0.2)(L3)

    L4 = Dense(128, activation='relu')(L3_D)
    L4_D = Dropout(0.2)(L4)

    L5 = Dense(64, activation='relu')(L4_D)
    L5_D = Dropout(0.2)(L5)

    L6 = Dense(32, activation='relu')(L5_D)
    L6_D = Dropout(0.2)(L6)

    L7 = Dense(8, activation='relu')(L6_D)

    prediction = Dense(1, activation='sigmoid')(L7)  # Output layer

    # Create the model
    sequential_net = Model(inputs=inputs, outputs=prediction)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    sequential_net.compile(loss='binary_crossentropy',
                        optimizer=optimizer,
                        metrics=["accuracy", "mae", "mse", tf.keras.metrics.AUC()])

    return sequential_net

In [19]:
from sklearn.model_selection import KFold

Epoch_S = 10

train = []
lbls_train = []
valid = []
lbls_valid = []

for i , data in enumerate(train_ds):
    embbed, lbl, smiles = data
    train.append(embbed[0])
    lbls_train.append(lbl.tolist())

for i , data in enumerate(test_ds):
    embbed, lbl, smiles = data
    valid.append(embbed[0])
    lbls_valid.append(lbl.tolist())

train = np.array(train).reshape(-1,1024,1)
lbls_train = np.array(lbls_train)

valid = np.array(valid).reshape(-1,1024,1)
lbls_valid = np.array(lbls_valid)

# create neural network model
sequential_net = sequential_model()
history = History()
P = sequential_net.fit(train, lbls_train, epochs = Epoch_S, batch_size = 128, callbacks=[history])

for j in range(100):
    C=1
    Before = int(P.history['accuracy'][-1]*100)
    for i in range(2,Epoch_S+1):
        if  int(P.history['accuracy'][-i]*100) == Before:
            C=C+1
        else:
            C=1
        Before=int(P.history['accuracy'][-i]*100)
        print(Before)
    if C==Epoch_S:
        break
    P = sequential_net.fit(train, lbls_train, epochs = Epoch_S, batch_size = 128, callbacks=[history])
print(j+1)

score  = sequential_net.evaluate(valid, lbls_valid, verbose=1)

Epoch 1/10
193/193 [==============================] - 16s 70ms/step - loss: 0.6127 - accuracy: 0.6660 - mae: 0.4183 - mse: 0.2113 - auc_1: 0.6593
Epoch 2/10
193/193 [==============================] - 16s 84ms/step - loss: 0.5535 - accuracy: 0.7202 - mae: 0.3731 - mse: 0.1860 - auc_1: 0.7528
Epoch 3/10
193/193 [==============================] - 16s 83ms/step - loss: 0.5181 - accuracy: 0.7452 - mae: 0.3451 - mse: 0.1716 - auc_1: 0.7911
Epoch 4/10
193/193 [==============================] - 9s 44ms/step - loss: 0.4831 - accuracy: 0.7725 - mae: 0.3175 - mse: 0.1577 - auc_1: 0.8226
Epoch 5/10
193/193 [==============================] - 10s 51ms/step - loss: 0.4549 - accuracy: 0.7864 - mae: 0.2971 - mse: 0.1473 - auc_1: 0.8442
Epoch 6/10
193/193 [==============================] - 9s 49ms/step - loss: 0.4329 - accuracy: 0.8035 - mae: 0.2793 - mse: 0.1386 - auc_1: 0.8607
Epoch 7/10
193/193 [==============================] - 9s 45ms/step - loss: 0.4123 - accuracy: 0.8125 - mae: 0.2654 - mse: 0.13

In [20]:
score

[0.740490198135376,
 0.8345534205436707,
 0.17564088106155396,
 0.13652780652046204,
 0.8767368197441101]